In [3]:
from nltk.corpus import sentiwordnet as swn
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [ ]:
class SentiWn(object):
    
    def __init__(self, strategy='sum', use_pos=False):
        self.strategy = strategy
        self.pos = use_pos
        self.pos_map = {
            'NOUN': 'n',
            'ADJ': 'a',
            'VERB': 'v',
            'ADV': 'r'
        }
        self.strategy_map = {'sum': self._simple_sum}
        
    def _simple_sum(self,  text):
        s = np.zeros(3)
        for token, pos in text:
            if self.pos:
                try:
                    synsets = list(swn.senti_synsets(token, self.pos_map[pos]))
                except KeyError:
                    pass
            else:
                synsets = list(swn.senti_synsets(token))
            for syn in synsets:
                p, n, o = syn.pos_score(), syn.neg_score(), syn.obj_score()
                s[0] += p
                s[1] += n
                s[2] += o
        return s
    
    def predict(self, docs):
        try:
            score_function = self.strategy_map[self.strategy]
        except KeyError:
            raise Exception('{} strategy not yet available'.format(self.strategy))
        y_pred = np.zeros(len(docs))
        doc_scores = []
        for doc in docs:
            doc_scores.append(score_function(doc))
        S = MinMaxScaler().fit_transform(np.array(doc_scores))
        pred = S.argmax(axis=1)
        for i, p in enumerate(pred):
            if p == 0:
                y_pred[i] = 1
            elif p == 1:
                y_pred[i] = -1
            else:
                pass
        return y_pred